## Importing Required Libraries

In [2]:
%pip install torch
%pip install torchvision

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets

from torchvision.transforms import ToTensor

## Download Dataset and Create Data Loader


In [4]:
#Download training data from MNIST datasets
training_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

#Download the test data from open datasets
test_data = datasets.MNIST(
    root = "data",
    train=False,
    download= True,
    transform = ToTensor(),
)

batch_size = 64

# Create data loaders to iterate over data
train_dataloader =  DataLoader(training_data, batch_size= batch_size)
test_dataloader = DataLoader(test_data, batch_size = batch_size)

print("Training data size:", len(train_dataloader)* batch_size)
print("Test data size:", len(test_dataloader) * batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, c, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break


Training data size: 60032
Test data size: 10048
Shape of X [N, c, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


## Define Model

In [5]:
# Get device for training
device = torch.device(
    "cuda" if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() # for apple mps is there
    else "cpu"
)

# device = torch.device("cuda:0")
print(f"Using {device} device")

# Define Model
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
        nn.Linear(input_size, hidden_size),
        nn.ReLU(),
        nn.Linear(hidden_size, hidden_size),
        nn.ReLU(),
        nn.Linear(hidden_size, num_classes)
        )
    def forward(self, image_tensor):
        image_tensor = self.flatten(image_tensor)
        logits = self.linear_relu_stack(image_tensor)
        return logits

input_size = 28 * 28
hidden_size = 512
num_classes = 10

model = NeuralNetwork(input_size, hidden_size, num_classes).to(device)
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


## Training Loop

In [6]:
# Define our learning rate, loss function and optimizer
learning_rate = 1e-3 #0.001
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

# Let's define our training function
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()

    for batch_num, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Forward pass to compute prediction
        pred = model(X)
        # Compute prediction error using loss function
        loss = loss_fn(pred, y)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch_num > 0 and batch_num % 100 == 0:
            loss, current = loss.item(), batch_num * len(X)
            print(f"loss: {loss: >7f} [{current:>5d}/{size: >5d}]")
        

## Test Loop

In [10]:
# Our test function
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    for X, y in dataloader:
        X, y = X.to(device), y.to(device)
        pred = model(X)
        test_loss += loss_fn(pred, y).item()
        correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100 * correct): >0.1f}%, Avg Loss: {test_loss: >8f} \n")

## Train the model

In [11]:
# let's run training
epochs = 5
for t in range(epochs):
    print(f"Epoch {t +1} \n ------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1 
 ------------------
loss: 0.105953 [ 6400/60000]
loss: 0.083715 [12800/60000]
loss: 0.107624 [19200/60000]
loss: 0.046807 [25600/60000]
loss: 0.123187 [32000/60000]
loss: 0.052951 [38400/60000]
loss: 0.113079 [44800/60000]
loss: 0.120215 [51200/60000]
loss: 0.117213 [57600/60000]
Test Error: 
 Accuracy: 96.9%, Avg Loss: 0.102425 

Epoch 2 
 ------------------
loss: 0.079620 [ 6400/60000]
loss: 0.027340 [12800/60000]
loss: 0.083875 [19200/60000]
loss: 0.051434 [25600/60000]
loss: 0.072921 [32000/60000]
loss: 0.023995 [38400/60000]
loss: 0.057088 [44800/60000]
loss: 0.073045 [51200/60000]
loss: 0.048602 [57600/60000]
Test Error: 
 Accuracy: 97.3%, Avg Loss: 0.100518 

Epoch 3 
 ------------------
loss: 0.063219 [ 6400/60000]
loss: 0.017306 [12800/60000]
loss: 0.209400 [19200/60000]
loss: 0.022043 [25600/60000]
loss: 0.059850 [32000/60000]
loss: 0.040782 [38400/60000]
loss: 0.027792 [44800/60000]
loss: 0.111472 [51200/60000]
loss: 0.030869 [57600/60000]
Test Error: 
 Accuracy: 97

## Save the model and make predictions

In [13]:
# Save our model parameters
torch.save(model.state_dict(), "ml_with_pytorch_model.pth")
print("Saved PyTorch Model State to ml_with_pytorch_model.pth")

# Load the saved modelparameters into a new instance of the model
model  = NeuralNetwork(input_size, hidden_size, num_classes).to(device)
model.load_state_dict(torch.load("ml_with_pytorch_model.pth"))

#Inference using the new model instance
model.eval()
for i in range(10):
    x, y = test_data[i][0], test_data[i][1]

    x = x.to(device)
    pred = model(x)
    predicted, actual = pred[0].argmax(0).item(),y
    print(f'Predicted: "{predicted}", Actual: "{actual}')

Saved PyTorch Model State to ml_with_pytorch_model.pth
Predicted: "7", Actual: "7
Predicted: "2", Actual: "2
Predicted: "1", Actual: "1
Predicted: "0", Actual: "0
Predicted: "4", Actual: "4
Predicted: "1", Actual: "1
Predicted: "4", Actual: "4
Predicted: "9", Actual: "9
Predicted: "5", Actual: "5
Predicted: "9", Actual: "9
